<a href="https://colab.research.google.com/github/quynhu-d/TTS_FastSpeech/blob/overfit_batch/Clean_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [ ]:
!git clone https://github.com/quynhu-d/TTS_FastSpeech

Cloning into 'TTS_FastSpeech'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 6), reused 41 (delta 4), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [ ]:
%cd TTS_FastSpeech/

/content/TTS_FastSpeech


In [ ]:
!pip install -r ./requirements.txt

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 298 kB 45.7 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 44.0 MB/s 
     |████████████████████████████████| 180 kB 46.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 132 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 35.7 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 243 kB 40.2 MB/s 
     |████████████████████████████████| 271 kB 46.0 MB/s 
     |████████████████████████████████| 192 kB 48.9 MB/s 
     |████████████████████████████████| 160 kB 46.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=468e7ffe2d58affb7edddcc24027ec2a938632bc30e4e922e40c9b45efb8b539
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb08

In [ ]:
from model import FastSpeech, FastSpeechConfig

# Data

In [ ]:
from data.lj_dataset import *

In [ ]:
from data.data_utils import *

In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xjf LJSpeech-1.1.tar.bz2

--2021-12-01 15:22:32--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  90.6MB/s    in 38s     

2021-12-01 15:23:10 (69.0 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [ ]:
from typing import Tuple, Dict, Optional, List, Union
from itertools import islice

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
from featurizer.mel_config import *
from featurizer.mel_spectrogram import *

# Model
Параметры в основном такие же, как в статье, но пока позьмём модель чуть поменьше, с трансформерами по 3 блока в энкодере и декодере (вместо 6).

In [ ]:
# Fast Speech model configuration
fconfig = FastSpeechConfig()
fconfig.d_model = 384
fconfig.d_k = 384
fconfig.d_v = 384
fconfig.n_dec = fconfig.n_enc = 3
print(fconfig)
fs_model = FastSpeech(51, fconfig)

FastSpeechConfig(d_model=384, conv_hid_sz=1536, kernel_sz=3, d_k=384, d_v=384, dropout_rate=0.1, n_enc=3, n_dec=3, n_heads=2, max_phoneme_len=5000, max_mel_len=5000, n_mels=80)


In [ ]:
device = 

## Vocoder

In [ ]:
!git clone https://github.com/NVIDIA/waveglow.git
!pip install googledrivedownloader

Cloning into 'waveglow'...
remote: Enumerating objects: 190, done.
remote: Total 190 (delta 0), reused 0 (delta 0), pack-reused 190
Receiving objects: 100% (190/190), 435.41 KiB | 4.11 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(
    file_id='1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF',
    dest_path='./waveglow_256channels_universal_v5.pt'
)

In [ ]:
# import warnings
# import sys
# sys.path.append('waveglow/')

# warnings.filterwarnings('ignore')
import torch.nn as nn

class Vocoder(nn.Module):

    def __init__(self):
        super(Vocoder, self).__init__()

        model = torch.load('../waveglow_256channels_universal_v5.pt', map_location='cpu')[
            'model']
        self.net = model.remove_weightnorm(model)

    @torch.no_grad()
    def inference(self, spect: torch.Tensor):
        spect = self.net.upsample(spect)

        # trim the conv artifacts
        time_cutoff = self.net.upsample.kernel_size[0] - \
            self.net.upsample.stride[0]
        spect = spect[:, :, :-time_cutoff]

        spect = spect.unfold(2, self.net.n_group, self.net.n_group) \
            .permute(0, 2, 1, 3) \
            .contiguous() \
            .flatten(start_dim=2) \
            .transpose(-1, -2)

        # generate prior
        audio = torch.randn(spect.size(0), self.net.n_remaining_channels, spect.size(-1)) \
            .to(spect.device)

        for k in reversed(range(self.net.n_flows)):
            n_half = int(audio.size(1) / 2)
            audio_0 = audio[:, :n_half, :]
            audio_1 = audio[:, n_half:, :]

            output = self.net.WN[k]((audio_0, spect))

            s = output[:, n_half:, :]
            b = output[:, :n_half, :]
            audio_1 = (audio_1 - b) / torch.exp(s)
            audio = torch.cat([audio_0, audio_1], 1)

            audio = self.net.convinv[k](audio, reverse=True)

            if k % self.net.n_early_every == 0 and k > 0:
                z = torch.randn(
                    spect.size(0), self.net.n_early_size, spect.size(2),
                    device=spect.device
                )
                audio = torch.cat((z, audio), 1)

        audio = audio.permute(0, 2, 1) \
            .contiguous() \
            .view(audio.size(0), -1)

        return audio

In [ ]:
%cd waveglow/
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocoder = Vocoder().to(device).eval()
%cd ..

[Errno 2] No such file or directory: 'waveglow/'
/content/TTS_FastSpeech/waveglow
/content/TTS_FastSpeech


# Overfitting

Оптимизатор -- Adam с параметрами для трансформеров (без планировщика)

In [ ]:
dataset = LJSpeechDataset('.')

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=3, collate_fn=LJSpeechCollator())

In [ ]:
iter(train_dataloader).next()

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: quynhu_d (use `wandb login --relogin` to force relogin)


True

In [ ]:
import torch.nn.functional as F
from model.aligner import GraphemeAligner
from model.fastspeech import FastSpeech
import numpy as np
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FastSpeech(51, fconfig).to(device)
print(model)
aligner = GraphemeAligner().to(device)
optimizer = torch.optim.Adam(model.parameters(), 3e-4, (.9,.98))
n_epochs = 30000
wandb.init(name='overfit_batch')
featurizer = MelSpectrogram(MelSpectrogramConfig()).to(device)
for i in range(n_epochs):
    for batch in train_dataloader:
        batch.to(device)
        optimizer.zero_grad()
        batch.mel = featurizer(batch.waveform)
        durations = aligner(batch.waveform, batch.waveform_length, batch.transcript)
        durations *= batch.mel.size(-1)
        batch.durations = durations.round()
        batch.to(device)
        output = model(batch)
        batch.to(device)
        # print(batch.duration_preds.sum(1), batch.mel.size(-1), batch.durations.sum(1))
        dp_loss = F.mse_loss(batch.duration_preds, batch.durations)
        print(output.size(), batch.mel.size())
        sz_diff = np.abs(batch.mel.size(-1) - output.size(-1))
        if batch.mel.size(-1) > output.size(-1):
            print('padding batch')
            output = F.pad(output, (0, sz_diff, 0, 0, 0, 0), "constant", MelSpectrogramConfig.pad_value)
        else:
            print('padding mel')
            batch.mel = F.pad(batch.mel, (0, sz_diff, 0, 0, 0, 0), "constant", MelSpectrogramConfig.pad_value)        
        print(batch.mel.shape, output.shape)
        mel_loss = F.mse_loss(output, batch.mel)
        loss = mel_loss + dp_loss
        loss.backward()
        optimizer.step()
        wav = vocoder.inference(output.to(device))
        idx = np.random.randint(batch.mel.shape[0])
        wandb.log({
            'loss': loss,
            'mel_loss': mel_loss,
            'dp_loss': dp_loss,
            'mel': wandb.Image(batch.mel[idx]),
            'mel_pred': wandb.Image(output[idx]),
            'audio': wandb.Audio(batch.waveform[idx].detach().cpu().numpy(), sample_rate=MelSpectrogramConfig.sr),
            'audio_pred': wandb.Audio(wav[idx].detach().cpu().numpy(), sample_rate=MelSpectrogramConfig.sr),
            'step': i
        })
        break

FastSpeech(
  (emb_encoder): Embedding(51, 384)
  (phoneme_pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (phoneme_fft): Sequential(
    (0): FFTBlock(
      (pre_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (attention): MultiHeadAttention(
        (att_heads): ModuleList(
          (0): SelfAttention(
            (wq): Linear(in_features=384, out_features=384, bias=True)
            (wk): Linear(in_features=384, out_features=384, bias=True)
            (wv): Linear(in_features=384, out_features=384, bias=True)
          )
          (1): SelfAttention(
            (wq): Linear(in_features=384, out_features=384, bias=True)
            (wk): Linear(in_features=384, out_features=384, bias=True)
            (wv): Linear(in_features=384, out_features=384, bias=True)
          )
        )
        (w_out): Linear(in_features=768, out_features=384, bias=True)
      )
      (att_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (co

KeyboardInterrupt: ignored